In [ ]:
[slides](https://docs.google.com/presentation/d/19K9nVjuSOCrZGM6lmFeEEarTm2xZwDSiZEIzf-Ywr5o/edit?usp=sharing)

[python-ecdsa docs](https://github.com/warner/python-ecdsa)

# Signing our First Message with ECDSA

In [4]:
from ecdsa import SigningKey, SECP256k1

private_key = SigningKey.generate(curve=SECP256k1)
public_key = private_key.get_verifying_key()

message = b'I need water'
sig = private_key.sign(message)

In [5]:
public_key.verify(sig, message)

True

# Defining ECDSACoin

* A coin is just a list of transfers, just like with PNGCoin. 
    * Where transfers were photographs of signatures in PNGCoin, they are ECDSA digital signatures in ECDSACoin
* The `public_key` in the last transfer is who owns the coin
* To spend the coin append a new transfer. Use the public key of the person you are sending to, and sign it using your private key.

In [8]:
class Transfer:
    
    def __init__(self, signature, public_key):
        self.signature = signature
        self.public_key = public_key
        
class ECDSACoin:
    
    def __init__(self, transfers):
        self.transfers = transfers

In [7]:
# The usual suspects ... 
# SECP256k1 is a detail about the "magical multiplication" used under the covers

bank_private_key = SigningKey.generate(curve=SECP256k1)
bob_private_key = SigningKey.generate(curve=SECP256k1)
alice_private_key = SigningKey.generate(curve=SECP256k1)

bank_public_key = bank_private_key.get_verifying_key()
bob_public_key = bob_private_key.get_verifying_key()
alice_public_key = alice_private_key.get_verifying_key()

In [17]:
from utils import serialize

def issue(public_key):
    message = serialize(public_key)
    
    signature = bank_private_key.sign(message)
    
    transfer = Transfer(
        signature=signature,
        public_key=public_key
    )
    coin = ECDSACoin([transfer])
    return coin

# Validating the First Transfer

In [18]:
def validate(coin):
    transfer = coin.transfers[0]
    message = serialize(transfer.public_key)
    bank_public_key.verify(transfer.signature, message)

In [19]:
alice_coin = issue(alice_public_key)

In [20]:
validate(alice_coin)

In [24]:
message = serialize(bob_public_key)
sig = bob_private_key.sign(message)

transfer = Transfer(
    signature=sig,
    public_key=bob_public_key
)
shit_coin = ECDSACoin([transfer])
validate(shit_coin)

BadSignatureError: 

# Validating Subsequent Transfers

In [1]:
def transfer_message(signature, public_key):
    return serialize({
        "previous_signature" : signature,
        "next_owner_public_key" : public_key
    })

def validate(coin):
    transfer = coin.transfers[0]
    message = serialize(transfer.public_key)
    bank_public_key.verify(transfer.signature, message)
    
    previous_transfer = coin.transfers[0]
    for next_transfer in coin.transfers[1:]:
        message = transfer_message(previous_transfer.signature, next_transfer.public_key)
        previous_transfer.public_key.verify(next_transfer.signature, message)
        previous_transfer = next_transfer
        

# Serialization

# The Finished Product

[ecdsacoin.py](ecdsacoin.py)

In [21]:
import ecdsacoin

coin = ecdsacoin.issue(alice_public_key)
coin.validate()

alice_to_bob = Transfer(
    signature=alice_private_key.sign(transfer_message(coin.transfers[-1].signature, bob_public_key)),
    public_key=bob_public_key,
)

coin.transfers.append(alice_to_bob)
coin.validate()